In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from datetime import datetime
from random import randint

C:\Users\parkchangho\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
# load / generate some toy datasets
df = pd.read_csv('dataset/staphycoccus.txt', delimiter='\t').dropna()

columns = df.columns[1:-1]
n_input = len(columns)
n_classes = 1

x_data = np.array(df[columns])
x_data = x_data.astype(float)

y_data = np.array(df[df.columns[-1]])
y_data = y_data.astype(float)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
y_train = np.reshape(y_train, (-1, 1))
y_test = np.reshape(y_test, (-1, 1))

X = tf.placeholder(tf.float32, [None, n_input], name='X')
Y = tf.placeholder(tf.float32, [None, n_classes], name='Y')

def hiddenStr(n_hidden_1, n_hidden_2, n_hidden_3):
    return '' + str(n_hidden_1) + '_' + str(n_hidden_2) + '_' + str(n_hidden_3)
    
def makeModel(layer):
    n_hidden_1, n_hidden_2, n_hidden_3 = layer

    with tf.name_scope('layer1'):
        W1 = tf.Variable(tf.random_uniform([n_input, n_hidden_1], -1., 1.))
        b1 = tf.Variable(tf.zeros([n_hidden_1]))
        L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

    with tf.name_scope('layer2'):
        W2 = tf.Variable(tf.random_uniform([n_hidden_1, n_hidden_2], -1., 1.))
        b2 = tf.Variable(tf.zeros([n_hidden_2]))
        L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))

    with tf.name_scope('layer3'):
        W3 = tf.Variable(tf.random_uniform([n_hidden_2, n_hidden_3], -1., 1.))
        b3 = tf.Variable(tf.zeros([n_hidden_3]))
        L3 = tf.nn.relu(tf.add(tf.matmul(L2, W3), b3))

    with tf.name_scope('output'):
        W4 = tf.Variable(tf.random_uniform([n_hidden_3, n_classes], -1., 1.))
        b4 = tf.Variable(tf.zeros([n_classes]))
        model = tf.add(tf.matmul(L3, W4), b4)

    with tf.name_scope('optimizer_' + hiddenStr(*layer)):
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=model))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

    with tf.name_scope('prediction_' + hiddenStr(*layer)):
        predictions = tf.cast(tf.sigmoid(model) > 0.5, tf.float32)
        is_equal = tf.equal(predictions, Y)
        accuracy = tf.summary.scalar('accuracy', tf.reduce_mean(tf.cast(is_equal, tf.float32))*100)
        
    return optimizer, accuracy

In [ ]:
times = 1
epochs = 1000
total_batch = 400
batch_size = int(len(x_train) / total_batch)

with tf.Session() as sess:
    for i in range(times):
        layer = (randint(1, 300), randint(1, 300), randint(1, 300))
        print(hiddenStr(*layer) + ' Start!')
        merged = tf.summary.merge_all()
        #time = datetime.now().strftime("%Y%m%d-%H%M%S")
        train_writer = tf.summary.FileWriter('./logs/train', sess.graph)
        test_writer = tf.summary.FileWriter('./logs/test', sess.graph)

        optimizer, accuracy = makeModel(layer)
        init = tf.global_variables_initializer()
        sess.run(init)

        for epoch in range(epochs):
            for i in range(total_batch):
                from_i = i*batch_size
                to_i = (i+1)*batch_size
                batch_xs = x_train[from_i:to_i]
                batch_ys = y_train[from_i:to_i]
                sess.run(optimizer, feed_dict={X: batch_xs, Y: batch_ys})
            summary = sess.run(accuracy,  feed_dict={X: x_train, Y: y_train})
            train_writer.add_summary(summary, epoch)
            train_writer.flush()

            summary = sess.run(accuracy,  feed_dict={X: x_test, Y: y_test})
            test_writer.add_summary(summary, epoch)
            test_writer.flush()
            if epoch % 250 == 0:
                print(hiddenStr(*layer) + ' : ' + str(epoch/50) + '%')

        train_writer.close()
        test_writer.close()
        print(hiddenStr(*layer) + ' Finish.')

261_142_24 Start!
261_142_24 : 0.0%
